In [39]:
%matplotlib notebook
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from matplotlib import pyplot as plt
from sklearn.neighbors import KNeighborsRegressor
import seaborn as sns
from sklearn.model_selection import train_test_split

In [40]:
def normalization(data):
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    data = scaler.fit_transform(data)
    return data

In [144]:
data = pd.read_csv("dtrain.csv", delimiter=",")
data = data[data['geocode_lat'] != 0]
zeros = data[data['geocode_lat'] == 0]
data = shuffle(data)
data = data[data['price'] < 1000000000]

In [145]:
#choosing data
X = data[['house_type_0', 'house_type_иное', 'house_type_каркасно-камышитовый',
       'house_type_кирпичный', 'house_type_монолитный',
       'house_type_панельный', 'state_0', 'state_евроремонт', 'state_свободная планировка',
       'state_среднее', 'state_требует ремонта', 'state_хорошее',
       'state_черновая отделка', 'region_Алматы', 'region_Алматы, Алатауский р-н',
       'region_Алматы, Алмалинский р-н', 'region_Алматы, Ауэзовский р-н',
       'region_Алматы, Бостандыкский р-н', 'region_Алматы, Жетысуйский р-н',
       'region_Алматы, Жетысуский р-н', 'region_Алматы, Медеуский р-н',
       'region_Алматы, Наурызбайский р-н', 'region_Алматы, Турксибский р-н',
       'region_Казахстан', 'internet_0', 'internet_ADSL', 'internet_оптика', 'internet_проводной',
       'internet_через TV кабель', 'bathroom_0', 'bathroom_2 с/у и более', 'bathroom_нет',
       'bathroom_раздельный', 'bathroom_совмещенный', 'built_time', 'all_space', 'balcony',
          'phone', 'parking', 'furniture', 'at_the_hostel',
          'appartments_floor', 'building_floors', 'price_for_sqr_meter',
         'trngl_first_point', 'trngl_second_point', 'trngl_third_point']]
#X_geo = data[['geocode_lat', 'geocode_long']]
y = data['price']
#x_columns = X.columns
#x_index = X.index
# X = normalization(X)
#X = pd.DataFrame(X, index=x_index, columns=x_columns)
#X = pd.concat([X, X_geo], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size=0.6)

In [146]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor

In [147]:
model = GradientBoostingRegressor(n_estimators = (len(X.columns) * 2), learning_rate=0.2, 
                                  random_state=7, max_depth=10)
model2 = RandomForestRegressor(n_estimators = (len(X.columns) * 2), random_state=7)

In [148]:
model.fit(X_train, y_train)
model2.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=94, n_jobs=1, oob_score=False, random_state=7,
           verbose=0, warm_start=False)

In [149]:
print(model.score(X_test, y_test))
print(model2.score(X_test, y_test))

0.991061185188
0.987515164065


In [150]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
print("GBM Mean absolute error: ", mean_absolute_error(model.predict(X_test), y_test))
print("GBM Mean squared error: ", mean_squared_error(model.predict(X_test), y_test))
print("RFR Mean absolute error: ", mean_absolute_error(model2.predict(X_test), y_test))
print("RFR Mean squared error: ", mean_squared_error(model2.predict(X_test), y_test))

GBM Mean absolute error:  390948.131768
GBM Mean squared error:  9.65833257426e+12
RFR Mean absolute error:  396394.633391
RFR Mean squared error:  1.34897858537e+13


In [160]:
pred_ans = (model.predict(X_test) + model2.predict(X_test)) / 2

In [161]:
print("Averaged Mean absolute error: ", mean_absolute_error(pred_ans, y_test))
print("Averaged Mean squared error: ", mean_squared_error(pred_ans, y_test))

Averaged Mean absolute error:  362746.022229
Averaged Mean squared error:  8.92202250591e+12


In [178]:
def scorer(clf, X, y):
    return mean_absolute_error(clf.predict(X), y)

In [179]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, X, y, cv=5, scoring=scorer)
print(scores)

[ 647287.46596698  531422.9693698   388292.40442356  739497.87738714
  402441.26239303]


In [180]:
scores.mean()

541788.39590810216

In [181]:
scores.std()

136605.40101914611

In [174]:
a = data[data.index==14255]
ax = X[data.index==14255]

In [175]:
float((model2.predict(ax)[0] + model.predict(ax)[0]) / 2)

14034171.09062176

In [176]:
float(a['price'])

14000000.0

In [177]:
abs(((model2.predict(ax)[0] + model.predict(ax)[0]) / 2) - float(a['price']))

34171.090621760115

In [169]:
X_test.sample(10)

,house_type_0,house_type_иное,house_type_каркасно-камышитовый,house_type_кирпичный,house_type_монолитный,house_type_панельный,state_0,state_евроремонт,state_свободная планировка,state_среднее,...,phone,parking,furniture,at_the_hostel,appartments_floor,building_floors,price_for_sqr_meter,trngl_first_point,trngl_second_point,trngl_third_point
14190,0,0,0,0,0,1,0,0,0,1,...,0.0,0.0,0.0,0.0,4.0,5.0,3.272727e+05,0.121381,-0.139347,0.006596
14255,0,0,0,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,5.0,2.500000e+05,0.194196,-0.066532,0.079411
14316,0,0,0,0,0,1,0,0,0,0,...,1.0,1.0,1.0,0.0,4.0,5.0,3.887500e+05,0.294702,0.033974,0.179917
23148,0,0,0,0,0,1,0,0,0,1,...,1.0,1.0,0.5,0.0,1.0,4.0,2.383333e+05,0.098151,-0.162577,-0.016634
23601,0,0,0,0,1,0,1,0,0,0,...,1.0,1.0,0.5,0.0,1.0,2.0,1.043333e+06,0.113467,-0.147261,-0.001318
6740,0,0,0,0,1,0,0,1,0,0,...,1.0,1.0,1.0,0.0,5.0,7.0,5.923077e+05,0.095924,-0.164804,-0.018861
24371,0,0,0,1,0,0,0,0,0,0,...,1.0,1.0,0.0,0.0,2.0,5.0,4.000000e+05,0.121045,-0.139683,0.006260
11650,0,0,0,0,1,0,0,0,1,0,...,1.0,0.0,0.0,0.0,14.0,15.0,4.678571e+05,0.109838,-0.150890,-0.004947
11,0,0,0,0,1,0,0,1,0,0,...,1.0,1.0,1.0,0.0,7.0,7.0,4.077670e+05,0.175187,-0.085541,0.060402
9260,0,0,0,0,0,1,1,0,0,0,...,0.0,0.0,0.5,0.0,9.0,9.0,2.500000e+05,0.095725,-0.165003,-0.019060
